In [2]:
!which python

/home/user/jfayzullaev/stellar-clustering/.venv-vis/bin/python


In [3]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import normalized_mutual_info_score as NMI, adjusted_rand_score as ARI


In [4]:
KMEANS_FILE_TX   = "transactions/transactions_node2vec_kmeans_results.csv"
KMEANS_FILE_TL   = "trustlines/trustlines_node2vec_kmeans_results.csv"

In [5]:
NORM_LABELS_PATH = "~/stellar-clustering/network/labled-data/labels/label-normalization/labels_entities_normalized.csv"  

In [8]:
N_SPLITS     = 5
RANDOM_STATE = 42

In [ ]:
def overall_purity_comm(df_comm_name: pd.DataFrame, comm_col: str, name_col: str = "name") -> float:

    if df_comm_name.empty:
        return np.nan
    counts = df_comm_name.groupby([comm_col, name_col]).size().reset_index(name='cnt')
    totals = counts.groupby(comm_col)['cnt'].sum()
    max_per_comm = counts.groupby(comm_col)['cnt'].max()
    return float(max_per_comm.sum() / totals.sum())


In [ ]:
def load_kmeans_fixed_k(kmeans_file: str, k_col: str) -> pd.DataFrame:

    if not os.path.exists(kmeans_file):
        raise FileNotFoundError(f"no file: {kmeans_file}")
    df = pd.read_csv(kmeans_file)


    df = df[['account_id', k_col]].dropna().drop_duplicates()
    df = df.rename(columns={k_col: 'cluster'})


    try:
        df['account_id'] = df['account_id'].astype(int)
    except Exception:
        df['account_id'] = df['account_id'].astype(str)
    return df

In [ ]:
def evaluate_fixed_kmeans_cv(
    labels_path: str,
    clusters_df: pd.DataFrame,
    label_col: str = "name",
    n_splits: int = 5,
    random_state: int = 42
):



    labels = (pd.read_csv(labels_path)
                .dropna(subset=['account_id', label_col])
                .drop_duplicates(subset=['account_id'])
                .rename(columns={label_col: 'name'}))

    # sync dtype
    try:
        labels['account_id'] = labels['account_id'].astype(int)
        clu = clusters_df.copy()
        clu['account_id'] = clu['account_id'].astype(int)
    except Exception:
        labels['account_id'] = labels['account_id'].astype(str)
        clu = clusters_df.copy()
        clu['account_id'] = clu['account_id'].astype(str)

    joined = labels.merge(clu, on='account_id', how='inner')

    n_labeled = len(labels)
    n_joined  = len(joined)
    coverage  = (n_joined / n_labeled) if n_labeled else 0.0
    if n_joined == 0:
        raise ValueError("No labeled accounts found in this K-Means file after join.")

    le = LabelEncoder()
    y_all = le.fit_transform(joined['name'].values)
    X_ids = joined['account_id'].values

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_ids, y_all), start=1):
        df_tr = joined.iloc[tr_idx].copy()
        df_te = joined.iloc[te_idx].copy()

        y_true_tr = le.transform(df_tr['name'])
        y_pred_tr = df_tr['cluster'].values
        nmi_tr = NMI(y_true_tr, y_pred_tr)
        ari_tr = ARI(y_true_tr, y_pred_tr)
        purity_tr = overall_purity_comm(df_tr[['cluster', 'name']], comm_col='cluster', name_col='name')

        y_true_te = le.transform(df_te['name'])
        y_pred_te = df_te['cluster'].values
        nmi_te = NMI(y_true_te, y_pred_te)
        ari_te = ARI(y_true_te, y_pred_te)
        purity_te = overall_purity_comm(df_te[['cluster', 'name']], comm_col='cluster', name_col='name')

        rows.append({
            'fold': fold,
            'n_train': len(df_tr),
            'n_test': len(df_te),
            'train_frac': len(df_tr) / len(joined),
            'NMI_train': nmi_tr,
            'ARI_train': ari_tr,
            'Purity_train': purity_tr,
            'NMI_test': nmi_te,
            'ARI_test': ari_te,
            'Purity_test': purity_te,
        })

    per_fold_df = pd.DataFrame(rows)

    averages = {
        'Avg_NMI_train': float(np.nanmean(per_fold_df['NMI_train'].values)),
        'Avg_ARI_train': float(np.nanmean(per_fold_df['ARI_train'].values)),
        'Avg_Purity_train': float(np.nanmean(per_fold_df['Purity_train'].values)),
        'Avg_NMI_test': float(np.nanmean(per_fold_df['NMI_test'].values)),
        'Avg_ARI_test': float(np.nanmean(per_fold_df['ARI_test'].values)),
        'Avg_Purity_test': float(np.nanmean(per_fold_df['Purity_test'].values)),
        'Avg_train_frac': float(np.nanmean(per_fold_df['train_frac'].values)),
    }

    coverage_info = {
        'n_labeled': int(n_labeled),
        'n_joined': int(n_joined),
        'coverage': float(coverage),
    }

    return per_fold_df, averages, coverage_info



## Transactions

In [6]:
K_COL = "kmeans_10" 

In [ ]:
clusters_df = load_kmeans_fixed_k(KMEANS_FILE_TX, K_COL)

In [13]:
norm_labels_path = os.path.expanduser(NORM_LABELS_PATH)

In [14]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_kmeans_cv(
    labels_path=norm_labels_path,
    clusters_df=clusters_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
print(f"\n K-Means {K_COL} | Normalized Labels per fold metrics")
display(norm_per_fold)


 K-Means K 10 | Normalized Labels per fold metrics


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,317,80,0.798489,0.286979,0.075387,0.716088,0.290929,0.070198,0.712500
1,2,317,80,0.798489,0.279752,0.084683,0.716088,0.283272,0.031752,0.712500
2,3,318,79,0.801008,0.276451,0.081237,0.713836,0.318803,0.050034,0.734177
3,4,318,79,0.801008,0.273606,0.077922,0.713836,0.291989,0.047189,0.721519
4,5,318,79,0.801008,0.260998,0.058012,0.716981,0.360715,0.153852,0.734177


In [ ]:
print(f"\n K-Means {K_COL} | Normalized Labels avgs")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")



 K-Means K 10 | Normalized Labels avgs
Avg_NMI_train: 0.275557
Avg_ARI_train: 0.075448
Avg_Purity_train: 0.715366
Avg_NMI_test: 0.309141
Avg_ARI_test: 0.070605
Avg_Purity_test: 0.722975
Avg_train_frac: 0.800000
Coverage: 4.90%  (397/8102)


In [17]:
TX_PATH = 'transactions/cross-validation'

In [ ]:
norm_per_fold.to_csv(f"{TX_PATH}/tx_cv_{K_COL}_norm_per_fold.csv", index=False)
pd.DataFrame([{
    **norm_avg,
    **norm_cov,
    "k_col": K_COL,
    "source_file": os.path.basename(KMEANS_FILE_TX)
}]).to_csv(f"{TX_PATH}/tx_cv_{K_COL}_norm_summary.csv", index=False)
print(f"Saved to {TX_PATH}")

Saved to transactions/cross-validation


## Trustlines

In [21]:
K_COL = "kmeans_10" 

In [ ]:
clusters_df = load_kmeans_fixed_k(KMEANS_FILE_TL, K_COL)

In [23]:
norm_labels_path = os.path.expanduser(NORM_LABELS_PATH)

In [24]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_kmeans_cv(
    labels_path=norm_labels_path,
    clusters_df=clusters_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
print(f"\n K-Means {K_COL} | Normalized Labels per fold metrics")
display(norm_per_fold)


 K-Means K 10 | Normalized Labels per fold metrics


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,111,28,0.798561,0.214025,0.045690,0.873874,0.303978,0.052332,0.892857
1,2,111,28,0.798561,0.216471,0.038105,0.873874,0.349814,0.090107,0.892857
2,3,111,28,0.798561,0.233293,0.050904,0.873874,0.240703,0.027607,0.857143
3,4,111,28,0.798561,0.202676,0.043589,0.873874,0.280326,0.061278,0.857143
4,5,112,27,0.805755,0.228847,0.048198,0.857143,0.175390,0.040699,0.925926


In [ ]:
print(f"\n K-Means {K_COL} | Normalized Labels avgs")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")



 K-Means K 10 | Normalized Labels avgs
Avg_NMI_train: 0.219063
Avg_ARI_train: 0.045297
Avg_Purity_train: 0.870528
Avg_NMI_test: 0.270042
Avg_ARI_test: 0.054405
Avg_Purity_test: 0.885185
Avg_train_frac: 0.800000
Coverage: 1.72%  (139/8102)


In [27]:
TL_PATH = 'trustlines/cross-validation'

In [ ]:
norm_per_fold.to_csv(f"{TL_PATH}/tx_cv_{K_COL}_norm_per_fold.csv", index=False)
pd.DataFrame([{
    **norm_avg,
    **norm_cov,
    "k_col": K_COL,
    "source_file": os.path.basename(KMEANS_FILE_TL)
}]).to_csv(f"{TL_PATH}/tx_cv_{K_COL}_norm_summary.csv", index=False)
print(f"Saved to {TL_PATH}")

Saved to trustlines/cross-validation
